<a href="https://colab.research.google.com/github/LuisFTacla/WDW_Attendance_Calculator/blob/main/Tratamento_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# IMPORTANDO BIBLIOTECAS
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

# LENDO URI
uri_01 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/splash_mountain.csv'
uri_02 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/pirates_of_caribbean.csv'
uri_03 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/7_dwarfs_train.csv'
uri_04 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/soarin.csv'
uri_05 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/spaceship_earth.csv'
uri_06 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/rock_n_rollercoaster.csv'
uri_07 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/toy_story_mania.csv'
uri_08 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/slinky_dog.csv'
uri_09 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/alien_saucers.csv'
uri_10 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/kilimanjaro_safaris.csv'
uri_11 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/dinosaur.csv'
uri_12 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/expedition_everest.csv'
uri_13 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/flight_of_passage.csv'
uri_14 = 'https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Datasets/Rides/Raw_Data/navi_river.csv'

# LENDO ARQUIVOS
mk_01 = pd.read_csv(uri_01)
mk_02 = pd.read_csv(uri_02)
mk_03 = pd.read_csv(uri_03)
ep_01 = pd.read_csv(uri_04)
ep_02 = pd.read_csv(uri_05)
hs_01 = pd.read_csv(uri_06)
hs_02 = pd.read_csv(uri_07)
hs_03 = pd.read_csv(uri_08)
hs_04 = pd.read_csv(uri_09)
ak_01 = pd.read_csv(uri_10)
ak_02 = pd.read_csv(uri_11)
ak_03 = pd.read_csv(uri_12)
ak_04 = pd.read_csv(uri_13)
ak_05 = pd.read_csv(uri_14)

# DICIONARIO COM NOMES DAS ATRACOES
nome_das_atracoes = {1: 'Splash Mountain',
                     2: 'Pirates of Caribbean',
                     3: 'Seven Dwarfs Mine Train',
                     4: 'Soarin',
                     5: 'Spaceship Earth',
                     6: 'Rock n Roller Coaster',
                     7: 'Toy Story Mania',
                     8: 'Slinky Dog Dash',
                     9: 'Alien Saucers',                
                     10: 'Kilimanjaro Safari',
                     11: 'Dinosaur',
                     12: 'Expedition Everest',
                     13: 'Avatar: Flight of Passage',
                     14: 'Navi River'}

# DICIONARIO COM DATAFRAMES
datasets_das_atracoes = {1: mk_01,
                         2: mk_02,
                         3: mk_03,
                         4: ep_01,
                         5: ep_02,
                         6: hs_01,
                         7: hs_02,
                         8: hs_03,
                         9: hs_04,
                         10: ak_01,
                         11: ak_02,
                         12: ak_03,
                         13: ak_04,
                         14: ak_05}

# FUNCOES PARA O TRATAMENTO DOS DADOS
def remove_sactmin(dados):
  dados.drop(columns = ['SACTMIN'], inplace = True)
  return dados

def remove_dados_ausentes(dados):
  dados.dropna(inplace= True)
  return dados

def to_datetime(dados):
  dados['data_e_hora'] = pd.to_datetime(dados['datetime'])
  return dados

def cria_coluna_tempo_de_fila(dados):
  dados['tempo_de_fila'] = dados['SPOSTMIN']
  return dados

def remove_colunas(dados):
  dados.drop(columns = ['SPOSTMIN', 'date', 'datetime'], inplace = True)
  return dados

def reset_index(dados):
  dados.reset_index(drop = True, inplace = True)
  return dados

def coluna_apagar(dados):
  dados['apagar'] = 0
  return dados

def tratamento_dados(dados):
  remove_sactmin(dados)
  remove_dados_ausentes(dados)
  to_datetime(dados)
  cria_coluna_tempo_de_fila(dados)
  remove_colunas(dados)
  reset_index(dados)
  coluna_apagar(dados)

# EXECUTANDO AS FUNCOES PARA CADA ATRACAO
for i in range(1,15):
  tratamento_dados(datasets_das_atracoes[i])

# FILTRANDO DADOS
for i in range(1,15):
  filtro = datasets_das_atracoes[i].tempo_de_fila >= 0
  datasets_das_atracoes[i] = datasets_das_atracoes[i][filtro]

# CRIANDO DATAFRAME BASE
bases = {}
instantes_5_em_5 = pd.date_range(start='1/1/2015', periods = 525888, 
                                 freq = '5min')
for i in range(1,15): 
  bases[i] = pd.DataFrame(index = range(0,525888), columns = ['data_e_hora', 
                                                              'tempo_de_fila', 
                                                              'apagar'])
  bases[i]['data_e_hora'] = instantes_5_em_5
  bases[i]['tempo_de_fila'] = 0
  bases[i]['apagar'] = 0
  bases[i]['observacao'] = bases[i].index
  bases[i].set_index('observacao', inplace = True)

# UNINDO DATAFRAMES AO BASE
datasets_das_atracoes_2 = {}

for i in range(1,15):
  datasets_das_atracoes_2[i] = bases[i].append(datasets_das_atracoes[i])

# ORDENANDO POR DATA/HORA
for i in range(1,15):
  datasets_das_atracoes_2[i].sort_values(by=['data_e_hora'], inplace = True)

# RESETANDO INDICE
for i in range(1,15):
  reset_index(datasets_das_atracoes_2[i])

# SELECIONANDO LINHAS PARA APAGAR
for j in range(1,15):
  dados_e_base_df = datasets_das_atracoes_2[j]
  for i in range(0,len(dados_e_base_df)):
    C1 = i != 0
    C2 = i != 1
    C3 = i != (len(dados_e_base_df) - 1)
    C4 = i != (len(dados_e_base_df) - 2)
    if C1 and C2 and C3 and C4:
      if (dados_e_base_df.tempo_de_fila[i] == 0):
        c1 = (dados_e_base_df.tempo_de_fila[i + 1] != 0)
        c2 = (dados_e_base_df.tempo_de_fila[i - 1] != 0)
        c3 = (dados_e_base_df.tempo_de_fila[i + 2] != 0)
        c4 = (dados_e_base_df.tempo_de_fila[i - 2] != 0)
        if c1 or c2 or c3 or c4:
          dados_e_base_df.apagar[i] = 1
    datasets_das_atracoes_2[j] = dados_e_base_df
  print('Opa! Terminei a leitura da atração {}!'.format(nome_das_atracoes[j]))
print('Pronto!! Terminei essa etapa!!')

# APAGANDO LINHAS
for i in range(1,15):
  filtro2 = datasets_das_atracoes_2[i].apagar == 0
  datasets_das_atracoes_2[i] = datasets_das_atracoes_2[i][filtro2]

# INTERPOLANDO TEMPOS DE FILA
datasets_das_atracoes_3 = {}

for i in range(1,15):

  datasets_das_atracoes_3[i] = bases[i]

  X_interp = bases[i].data_e_hora
  X = datasets_das_atracoes_2[i].data_e_hora
  Y = datasets_das_atracoes_2[i].tempo_de_fila
  
  Y_interp = np.interp(X_interp, X, Y)

  datasets_das_atracoes_3[i].tempo_de_fila = np.around(Y_interp, decimals = 0)

# SALVANDO OS DADOS TRATADOS
datasets_das_atracoes_3[1].to_csv((r'Splash_Mountain.csv'), index = False)
datasets_das_atracoes_3[2].to_csv((r'Pirates_of_Caribbean.csv'), index = False)
datasets_das_atracoes_3[3].to_csv((r'Seven_Dwarfs_Mine_Train.csv'), index = False)
datasets_das_atracoes_3[4].to_csv((r'Soarin.csv'), index = False)
datasets_das_atracoes_3[5].to_csv((r'Spaceship_Earth.csv'), index = False)
datasets_das_atracoes_3[6].to_csv((r'Rock_n_Roller_Coaster.csv'), index = False)
datasets_das_atracoes_3[7].to_csv((r'Toy_Story_Mania.csv'), index = False)
datasets_das_atracoes_3[8].to_csv((r'Slinky_Dog_Dash.csv'), index = False)
datasets_das_atracoes_3[9].to_csv((r'Alien_Saucers.csv'), index = False)
datasets_das_atracoes_3[10].to_csv((r'Kilimanjaro_Safari.csv'), index = False)
datasets_das_atracoes_3[11].to_csv((r'Dinosaur.csv'), index = False)
datasets_das_atracoes_3[12].to_csv((r'Expedition_Everest.csv'), index = False)
datasets_das_atracoes_3[13].to_csv((r'Avatar_Flight_of_Passage.csv'), index = False)
datasets_das_atracoes_3[14].to_csv((r'Navi_River.csv'), index = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Opa! Terminei a leitura da atração Splash Mountain!
Opa! Terminei a leitura da atração Pirates of Caribbean!
Opa! Terminei a leitura da atração Seven Dwarfs Mine Train!
Opa! Terminei a leitura da atração Soarin!
Opa! Terminei a leitura da atração Spaceship Earth!
Opa! Terminei a leitura da atração Rock n Roller Coaster!
Opa! Terminei a leitura da atração Toy Story Mania!
Opa! Terminei a leitura da atração Slinky Dog Dash!
Opa! Terminei a leitura da atração Alien Saucers!
Opa! Terminei a leitura da atração Kilimanjaro Safari!
Opa! Terminei a leitura da atração Dinosaur!
Opa! Terminei a leitura da atração Expedition Everest!
Opa! Terminei a leitura da atração Avatar: Flight of Passage!
Opa! Terminei a leitura da atração Navi River!
Pronto!! Terminei essa etapa!!
